## Install Dependencies

In [ ]:
!pip install llm2vec torch-geometric jsonlines

## Libraries

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import jsonlines
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import MinMaxScaler

from torch_geometric.nn import GATConv
import torch.nn.functional as F

from llm2vec import LLM2Vec
from torch_geometric.data import Data

## Load Model

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set paths for models and data
model_path = "/kaggle/input/nlp-deception/best_model.pth"
file_path = "/kaggle/input/nlp-deception/val.jsonl"

EMBEDDER_PATH = "McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp"
PEFT_PATH = "McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp-supervised"

# Set inference parameters
threshold = 0.5
hidden_dim = 512

In [ ]:
# Load the LLM2Vec model for generating embeddings
llm2vec_model = LLM2Vec.from_pretrained(
    EMBEDDER_PATH,
    peft_model_name_or_path=PEFT_PATH,
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)

In [ ]:
# Load the trained GNN model
def load_model(model_path, input_dim, hidden_dim):
    try:
        model = GATDeceptionClassifier(input_dim, hidden_dim).to(device)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()
        print(f"Model loaded successfully from {model_path}")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

In [ ]:
# Load dataset
def load_flattened_dataset(file_path):
    data = []
    with jsonlines.open(file_path) as reader:
        for game in reader:
            for i in range(len(game["messages"])):
                if game["sender_labels"][i] == "NOANNOTATION":
                    continue
                data.append({
                    "message": game["messages"][i],
                    "sender_label": int(game["sender_labels"][i] == False) if game["sender_labels"][i] != "NOANNOTATION" else None,
                    "receiver_label": game["receiver_labels"][i],
                    "speaker": game["speakers"][i],
                    "receiver": game["receivers"][i],
                    "abs_msg_idx": game["absolute_message_index"][i],
                    "rel_msg_idx": game["relative_message_index"][i],
                    "season": game["seasons"][i],
                    "year": game["years"][i],
                    "score": game["game_score"][i],
                    "score_delta": float(game["game_score_delta"][i]),
                    "game_id": game["game_id"],
                    "players": game["players"],
                    "message_length": len(game["messages"][i])
                })
    return pd.DataFrame(data)

In [ ]:
# Text cleaning function
def soft_clean(text):
    text = text.replace('\n', ' ').strip()
    return text

In [ ]:
# Load the dataset
df = load_flattened_dataset(file_path)
print(f"Loaded {len(df)} messages from {file_path}")
print(df.head())

## Create Embeddings

In [ ]:
def embed_messages(df, model):
    embeddings = []
    for msg in tqdm(df["message"], desc="Embedding messages"):
        emb = model.encode(msg)[0]  # Returns 1 vector
        embeddings.append(torch.tensor(emb, dtype=torch.float32))
    return torch.stack(embeddings)

In [ ]:
def create_embeddings(file: str, output_path: str):
    df = load_flattened_dataset(file)
    x = embed_messages(df, llm2vec_model)
    y = torch.tensor(df["sender_label"].values, dtype=torch.float32)
    torch.save((x, y), output_path)

In [ ]:
create_embeddings(file_path, '/kaggle/working/train_embeddings.pt')

## Graph from data

In [ ]:
# Extract metadata features
def extract_metadata(df):
    scaler = MinMaxScaler()
    metadata_features = df[["abs_msg_idx", "rel_msg_idx", "year", "score", 
                           "score_delta", "message_length"]].values
    metadata_features = scaler.fit_transform(metadata_features)
    return metadata_features

In [ ]:
# Extract player features
def player_features(df):
    players = np.unique(df[["speaker", "receiver"]].values.flatten())
    player_node_features = np.eye(len(players))
    return player_node_features, players

In [ ]:
# Pad player features to match message feature dimensions
def pad_player_features(player_features, padding_dim):
    return np.pad(player_features, ((0, 0), (0, padding_dim)), 'constant')

In [ ]:
# Build edge list for the graph
def build_bidirectional_edge_list(df, players):
    from sklearn.preprocessing import LabelEncoder
    
    edges = []
    num_messages = len(df)
    player_encoder = LabelEncoder().fit(players)
    player_offset = num_messages  # player nodes start after message nodes
    
    for i in range(num_messages - 1):
        # Temporal edges: message i <-> message i+1
        edges.append((i, i + 1))
        edges.append((i + 1, i))

    for i, row in df.iterrows():
        speaker_id = player_offset + player_encoder.transform([row["speaker"]])[0]
        receiver_id = player_offset + player_encoder.transform([row["receiver"]])[0]

        # Speaker <-> message
        edges.append((speaker_id, i))   # player → message
        edges.append((i, speaker_id))   # message → player

        # Receiver <-> message
        edges.append((receiver_id, i))
        edges.append((i, receiver_id))

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    return edge_index

## Model

In [ ]:
class GATDeceptionClassifier(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=512):
        super().__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=4, dropout=0.3)
        self.gat2 = GATConv(hidden_dim * 4, hidden_dim, heads=2, dropout=0.3)
        self.out = torch.nn.Linear(hidden_dim * 2, 1)
        self.out_dim = hidden_dim * 2

    def forward(self, x, edge_index):
        x = F.elu(self.gat1(x, edge_index))
        x = F.elu(self.gat2(x, edge_index))
        return self.out(x).squeeze()

In [ ]:
# Extract metadata features
metadata_features = extract_metadata(df)
# Create player node features
player_node_features, players = player_features(df)
# Combine embeddings with metadata
message_node_features = np.hstack([embeddings.cpu().numpy(), metadata_features])

In [ ]:
# Calculate padding for player features
message_dim = message_node_features.shape[1]
player_dim = player_node_features.shape[1]
padding_dim = message_dim - player_dim

In [ ]:
# Pad player features
player_node_features_padded = pad_player_features(player_node_features, padding_dim

In [ ]:
# Combine message and player features
x = torch.tensor(np.vstack([message_node_features, player_node_features_padded]), 
                 dtype=torch.float32)

In [ ]:
# Build edges
edge_index = build_bidirectional_edge_list(df, players)
# Create the graph
graph_data = Data(x=x, edge_index=edge_index)
graph_data = graph_data.to(device)

## Inference

In [ ]:
# Load the GNN model
input_dim = x.shape[1]
model = load_model(MODEL_PATH, input_dim, hidden_dim)

In [ ]:
# Run inference
def predict(model, data, threshold=0.5):
    model.eval()
    with torch.no_grad():
        logits = model(data.x, data.edge_index)
        probs = torch.sigmoid(logits)
        # Only take predictions for message nodes (not player nodes)
        message_probs = probs[:len(df)].cpu().numpy()
        predictions = (message_probs > threshold).astype(int)
    return message_probs, predictions

In [ ]:
# Get predictions
probabilities, predictions = predict(model, graph_data, THRESHOLD)

# Add predictions to the dataframe
df['deception_probability'] = probabilities
df['predicted_deceptive'] = predictions

print("Inference completed")
print(f"Found {predictions.sum()} potentially deceptive messages out of {len(df)}")

## Analyze Results

In [ ]:
# Calculate metrics if ground truth is available
if 'sender_label' in df.columns and not df['sender_label'].isna().all():
    valid_idx = ~df['sender_label'].isna()
    y_true = df.loc[valid_idx, 'sender_label'].astype(int).values
    y_pred = df.loc[valid_idx, 'predicted_deceptive'].values
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Truthful', 'Deceptive'], 
                yticklabels=['Truthful', 'Deceptive'])
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(f"{output_dir}/confusion_matrix.png")
    plt.show()

In [ ]:
# Distribution of deception probabilities
plt.figure(figsize=(10, 6))
sns.histplot(df['deception_probability'], bins=50, kde=True)
plt.axvline(x=THRESHOLD, color='red', linestyle='--', label=f'Threshold ({THRESHOLD})')
plt.title('Distribution of Deception Probabilities')
plt.xlabel('Deception Probability')
plt.ylabel('Frequency')
plt.legend()
plt.savefig(f"{output_dir}/probability_distribution.png")
plt.show()

In [ ]:
# Deception by player
plt.figure(figsize=(12, 6))
player_deception = df.groupby('speaker')['deception_probability'].mean().sort_values(ascending=False)
sns.barplot(x=player_deception.index, y=player_deception.values)
plt.title('Average Deception Probability by Player')
plt.xlabel('Player')
plt.ylabel('Avg. Deception Probability')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig(f"{output_dir}/player_deception.png")
plt.show()

In [ ]:
# Message length vs deception probability
plt.figure(figsize=(10, 6))
sns.scatterplot(x='message_length', y='deception_probability', data=df, alpha=0.6)
plt.title('Message Length vs. Deception Probability')
plt.xlabel('Message Length')
plt.ylabel('Deception Probability')
plt.savefig(f"{output_dir}/length_vs_deception.png")
plt.show()

In [ ]:
# Export predictions to CSV
output_path = f"{output_dir}/deception_predictions.csv"
df.to_csv(output_path, index=False)
print(f"Predictions saved to {output_path}")

In [ ]:
# Most highly deceptive messages
print("Top 10 most likely deceptive messages:")
top_deceptive = df.sort_values('deception_probability', ascending=False).head(10)
for i, row in top_deceptive.iterrows():
    print(f"Player: {row['speaker']} | Prob: {row['deception_probability']:.4f} | Message: {row['message'][:100]}...")